<a href="https://colab.research.google.com/github/woodword-0/NLP_Project_SJ/blob/main/TensorFlowKerasSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_hub as hub

Custom Tokenizer

In [ ]:
!wget --show-progress --continue -O /content/Oddesey.txt http://www.gutenberg.org/files/1727/1727-0.txt

In [ ]:
!head-n5 /content/Oddesey.txt
!echo "..."
!shuf -n5 /content/Oddesey.txt

Building an Embedder

In [ ]:
filepath = '/content/Oddesey.txt'

In [ ]:
with open(filepath) as f:
  Oddesey_text = f.read()

In [ ]:
Oddesey_text.shape

Tokenizer from Keras based on the text

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) #not a true embedding yet

In [ ]:
tokenizer.fit_on_texts([Oddesey_text])

In [ ]:
[encode] = np.array(tokenizer.texts_to_sequences([Oddesey_text]))

In [ ]:
tokenizer.texts_to_sequences(['king'])

[[26, 9, 7, 19]]

In [ ]:
tokenizer.texts_to_sequences(['Zeus'])

[[43, 2, 13, 8]]

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name = "imdb_reviews",
    split = ('train[:60%]', 'train[:20%]','test'), as_supervised = True
)

In [ ]:
tfds.list_builders()

Import an embedding layer

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [ ]:
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
#embedding layer
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype = tf.string, trainable= True) #trainable freezes weights turn on or off #Tokenizing Layer
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

Build a model for this

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation = 'relu')) #layers [20,16,1]
model.add(tf.keras.layers.Dense(1, activation = "sigmoid")) #here we build a simple classifier

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='Adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics= 'accuracy'
              ) #from logits takes into account probability

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs = 20, validation_data= validation_data.batch(512),verbose = 1)

Epoch 1/20
30/30 [==============================] - 3s 75ms/step - loss: 0.8992 - accuracy: 0.4593 - val_loss: 0.6459 - val_accuracy: 0.6256
Epoch 2/20
30/30 [==============================] - 3s 71ms/step - loss: 0.6069 - accuracy: 0.6769 - val_loss: 0.5136 - val_accuracy: 0.7798
Epoch 3/20
30/30 [==============================] - 3s 71ms/step - loss: 0.4895 - accuracy: 0.7937 - val_loss: 0.4128 - val_accuracy: 0.8380
Epoch 4/20
30/30 [==============================] - 3s 72ms/step - loss: 0.3881 - accuracy: 0.8549 - val_loss: 0.3406 - val_accuracy: 0.8792
Epoch 5/20
30/30 [==============================] - 3s 72ms/step - loss: 0.3232 - accuracy: 0.8889 - val_loss: 0.2932 - val_accuracy: 0.8972
Epoch 6/20
30/30 [==============================] - 3s 72ms/step - loss: 0.2794 - accuracy: 0.9024 - val_loss: 0.2595 - val_accuracy: 0.9088
Epoch 7/20
30/30 [==============================] - 3s 74ms/step - loss: 0.2423 - accuracy: 0.9179 - val_loss: 0.2356 - val_accuracy: 0.9178
Epoch 8/20
30

In [ ]:
results = model.evaluate(test_data.batch(512),verbose = 2) #test accuracy

49/49 - 2s - loss: 0.3392 - accuracy: 0.8639


In [ ]:
model.predict(["This was the best movie ever"])

array([[0.9293436]], dtype=float32)

In [ ]:
model.predict(["This was the worst movie ever"])

array([[0.01385111]], dtype=float32)

In [ ]:
#add the sigmoid
(model.predict(["This was the best movie ever"]) >0.5).astype('int32')

array([[1]], dtype=int32)

In [ ]:
(model.predict(["This was the worst movie ever"]) >0.5).astype('int32')

array([[0]], dtype=int32)

In [ ]:
(model.predict(["I hate to like this movie"]) >0.5).astype('int32')

array([[0]], dtype=int32)